In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import ldspec

In [2]:
# Reformat sumstats 
SUMSTATS_FILE='/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/sumstats/@.nomhc.sumstats.gz'
TRAIT_LIST=list(pd.read_csv(
    '/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001/UKBB_trait/trait_list_all.txt', header=None
)[0])
OUT_PATH='/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/ldsc_res/sumstats'

for trait in TRAIT_LIST:
    df_sumstats = pd.read_csv(SUMSTATS_FILE.replace('@', trait), sep="\t", index_col=None)
    ind_select = ~df_sumstats['Z'].isna()
    df_sumstats.loc[ind_select].to_csv(
        OUT_PATH+'/%s.nomhc.sumstats.gz' % trait, sep='\t', index=False
    )

In [2]:
# Reformat LD files
DATA_PATH='/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp'
ANNOT_FILE='/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001/baseline_annot/baseline_165annots_chr@.annot.gz'
SCORE_FILE=DATA_PATH+'/ldspec_score_file/baseline.c@_score.tsv.gz'
PGEN_FILE='/n/scratch3/users/j/jz286/imp_geno_chimp/ukb_imp_chr@_v3_chimp'

OUT_PATH='/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/ldsc_res/ld_file'

AN_list = [x for x in ldspec.util.read_annot(ANNOT_FILE.replace('@', '%d'%1), nrows=5) if x.startswith('AN:')]
print('n_annot=%d' % len(AN_list))

for CHR in [22]:
# for CHR in range(1,22):
    df_annot = ldspec.util.read_annot(ANNOT_FILE.replace('@', '%d'%CHR))
    df_score = pd.read_csv(SCORE_FILE.replace('@', '%d'%CHR), sep="\t", index_col=None)
    df_frq = ldspec.util.read_pgen(PGEN_FILE.replace("@", "%d" % CHR))["afreq"]
    print('df_annot %d' % df_annot.shape[0], 'df_score %d' % df_score.shape[0], 'df_frq %d' % df_frq.shape[0])
    
    df_score['MAF'] = df_frq["MAF"].astype(np.float32)
    df_score['CM'] = 0
    for AN in AN_list:
        df_score['LD:%s'%AN] = df_score['LD:%s'%AN].astype(np.float32)
    
    # .annot.gz
    df_annot[['CHR', 'BP', 'SNP', 'CM'] + AN_list].to_csv(
        OUT_PATH + '/bslLF165.%d.annot.gz' % CHR, sep='\t', index=False
    )
    # .@.l2.ldscore.gz
    df_score[['CHR', 'BP', 'SNP', 'CM', 'MAF'] + ['LD:%s'%x for x in AN_list]].to_csv(
        OUT_PATH + '/bslLF165.%d.l2.ldscore.gz' % CHR, sep='\t', index=False
    )
    # .M
    temp_str = ''
    for AN in AN_list:
        temp_str += '%.2f ' % df_annot[AN].sum()
    with open(OUT_PATH + '/bslLF165.%d.l2.M' % CHR, 'w') as f:
        f.write('%s\n' % temp_str[:-1])
    
    # weights.hm3_noMHC.9.l2.ldscore.gz
    temp_df = df_score[['CHR', 'SNP', 'BP', 'LD:AN:all']].copy()
    temp_df.columns = ['CHR', 'SNP', 'BP', 'L2']
    temp_df.to_csv(
        OUT_PATH + '/bslLF165.weights.%d.l2.ldscore.gz' % CHR, sep='\t', index=False
    )
    print('Finished CHR%d' % CHR)


n_annot=165
df_annot 199680 df_score 199680 df_frq 199680
Finished CHR22


In [32]:
temp_df = df_score[['CHR', 'SNP', 'BP', 'LD:AN:all']].copy()
temp_df.columns = ['CHR', 'SNP', 'BP', 'L2']
temp_df.to_csv(
    OUT_PATH + '/bslLF165.weights.%d.l2.ldscore.gz' % CHR, sep='\t', index=False
)

In [26]:
df_score['LD:AN:Backgrd_Selection_Stat_common']

0         0.004619
1         0.047587
2         0.048080
3         0.047960
4         0.034349
            ...   
199675    4.499467
199676    0.424114
199677    0.173055
199678    0.135685
199679    0.512254
Name: LD:AN:Backgrd_Selection_Stat_common, Length: 199680, dtype: float32

In [11]:
df_annot

,CHR,SNP,BP,CM,AN:Backgrd_Selection_Stat_common,AN:Backgrd_Selection_Stat_lf,AN:BivFlnk.flanking.500_common,AN:BivFlnk.flanking.500_lf,AN:BivFlnk_common,AN:BivFlnk_lf,...,AN:mbin8_common,AN:mbin9_common,AN:nucleotide_div_common,AN:nucleotide_div_lf,AN:recomb_rate_common,AN:recomb_rate_lf,AN:snpeff_nonsynonymous_variant_common,AN:snpeff_nonsynonymous_variant_lf,AN:snpeff_synonymous_variant_common,AN:snpeff_synonymous_variant_lf
0,22,rs587769434,16050527,0,0.000,0.0,False,False,False,False,...,False,False,0.000000,0.0,0.000000,0.0,False,False,False,False
1,22,rs62224609,16051249,0,0.006,0.0,False,False,False,False,...,False,False,0.559597,0.0,6.381581,0.0,False,False,False,False
2,22,rs62224614,16053862,0,0.006,0.0,False,False,False,False,...,False,False,0.449944,0.0,5.886115,0.0,False,False,False,False
3,22,rs7286962,16054454,0,0.006,0.0,False,False,False,False,...,False,False,0.430818,0.0,5.799526,0.0,False,False,False,False
4,22,22:16056406_C_T,16056406,0,0.000,0.0,False,False,False,False,...,False,False,0.000000,0.0,0.000000,0.0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199675,22,rs370652263,51237712,0,0.094,0.0,False,False,False,False,...,False,False,0.000000,0.0,0.000000,0.0,False,False,False,False
199676,22,rs541098394,51238318,0,0.000,0.0,False,False,False,False,...,False,False,0.000000,0.0,0.000000,0.0,False,False,False,False
199677,22,rs553081191,51238328,0,0.000,0.0,False,False,False,False,...,False,False,0.000000,0.0,0.000000,0.0,False,False,False,False
199678,22,rs149712012,51238394,0,0.000,0.0,False,False,False,False,...,False,False,0.000000,0.0,0.000000,0.0,False,False,False,False


In [4]:
df_annot

,CHR,SNP,BP,CM,AN:Backgrd_Selection_Stat_common,AN:Backgrd_Selection_Stat_lf,AN:BivFlnk.flanking.500_common,AN:BivFlnk.flanking.500_lf,AN:BivFlnk_common,AN:BivFlnk_lf,...,AN:mbin8_common,AN:mbin9_common,AN:nucleotide_div_common,AN:nucleotide_div_lf,AN:recomb_rate_common,AN:recomb_rate_lf,AN:snpeff_nonsynonymous_variant_common,AN:snpeff_nonsynonymous_variant_lf,AN:snpeff_synonymous_variant_common,AN:snpeff_synonymous_variant_lf
0,22,rs587769434,16050527,0,0.000,0.0,False,False,False,False,...,False,False,0.000000,0.0,0.000000,0.0,False,False,False,False
1,22,rs62224609,16051249,0,0.006,0.0,False,False,False,False,...,False,False,0.559597,0.0,6.381581,0.0,False,False,False,False
2,22,rs62224614,16053862,0,0.006,0.0,False,False,False,False,...,False,False,0.449944,0.0,5.886115,0.0,False,False,False,False
3,22,rs7286962,16054454,0,0.006,0.0,False,False,False,False,...,False,False,0.430818,0.0,5.799526,0.0,False,False,False,False
4,22,22:16056406_C_T,16056406,0,0.000,0.0,False,False,False,False,...,False,False,0.000000,0.0,0.000000,0.0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199675,22,rs370652263,51237712,0,0.094,0.0,False,False,False,False,...,False,False,0.000000,0.0,0.000000,0.0,False,False,False,False
199676,22,rs541098394,51238318,0,0.000,0.0,False,False,False,False,...,False,False,0.000000,0.0,0.000000,0.0,False,False,False,False
199677,22,rs553081191,51238328,0,0.000,0.0,False,False,False,False,...,False,False,0.000000,0.0,0.000000,0.0,False,False,False,False
199678,22,rs149712012,51238394,0,0.000,0.0,False,False,False,False,...,False,False,0.000000,0.0,0.000000,0.0,False,False,False,False


In [7]:
list(df_score.columns)

['CHR',
 'SNP',
 'BP',
 'E',
 'LD:AN:Backgrd_Selection_Stat_common',
 'LD:AN:Backgrd_Selection_Stat_lf',
 'LD:AN:BivFlnk.flanking.500_common',
 'LD:AN:BivFlnk.flanking.500_lf',
 'LD:AN:BivFlnk_common',
 'LD:AN:BivFlnk_lf',
 'LD:AN:CADD_g20_common',
 'LD:AN:CADD_g20_lf',
 'LD:AN:CTCF_Hoffman.flanking.500_common',
 'LD:AN:CTCF_Hoffman.flanking.500_lf',
 'LD:AN:CTCF_Hoffman_common',
 'LD:AN:CTCF_Hoffman_lf',
 'LD:AN:Coding_UCSC.flanking.500_common',
 'LD:AN:Coding_UCSC.flanking.500_lf',
 'LD:AN:Coding_UCSC_common',
 'LD:AN:Coding_UCSC_lf',
 'LD:AN:Conserved_LindbladToh.flanking.500_common',
 'LD:AN:Conserved_LindbladToh.flanking.500_lf',
 'LD:AN:Conserved_LindbladToh_common',
 'LD:AN:Conserved_LindbladToh_lf',
 'LD:AN:Conserved_Mammal_phastCons46way.flanking.500_common',
 'LD:AN:Conserved_Mammal_phastCons46way.flanking.500_lf',
 'LD:AN:Conserved_Mammal_phastCons46way_common',
 'LD:AN:Conserved_Mammal_phastCons46way_lf',
 'LD:AN:Conserved_Primate_phastCons46way.flanking.500_common',
 'LD:A